[Node 18: Synchronisation](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node18.html)

Navigation:

 **Next:** [Python Threads und GIL](node19.ipynb) **Up:** [Threads and Multi–Processing](node16.ipynb) **Previous:** [Die Thread Klasse](node17.ipynb)

##  Synchronisation
 In den bisherigen Beispielen laufen die Threads unabhängig  voneinander. Komplizierter wird es wenn sie auf gemeinsame Datenbereiche zugreifen, insbesondere wenn ein Thread schreibt und ein anderes liest.   Das folgende –etwas konstruierte– Beispiel illustriert das Problem  

---

In [ ]:
import threading 
  
class Counter(object):
    def __init__(self):
        self.num = 0
    def inc(self):
        self.num += 1

def thread_task(counter):
    for _ in range(100000):
        counter.inc()
  
# global counter
counter = Counter()

# create two threads 
t1 = threading.Thread(target = thread_task, args = (counter,))
t2 = threading.Thread(target = thread_task, args = (counter,))
  
# start threads 
t1.start() 
t2.start() 
  
# wait until threads finish their job 
t1.join() 
t2.join() 
  
# print result
print(counter.num) 



 Beide Threads greifen auf dasselbe   <font color=#008000> *Counter–Objekt*</font>  zu, d.h. sie benutzen diesselbe   <font color=#ff0000> **Member-variable**</font>  **self.num**.  
 
Abhängig vom zufälligen Ablauf erhalten wir das erwartete Ergebnis (200000) oder aber eine andere Zahl zwischen 100000 und 200000. Das Programm ist nicht–deterministisch, obwohl keine Zufallszahlen benutzt  werden... 

 <font color=#ff0000> **Probieren Sie's aus !**</font>  
 
Man kann im Beispiel die Variablen anders definieren/einsetzen und das spezifische Problem damit beheben, z.B. jeden Thread einen eigenen Counter geben. Das ist aber keine Lösung für den allgemeinen Fall.   

Echte Abhilfe bietet in Python (u.a.) der    <font color=#ff0000> **Locking**</font>  Mechanismus:  
*  <font color=#0000e6> ``lck=threading.Lock()``</font>  Objekt wird erzeugt 
* Aufruf von   <font color=#0000e6> ``lck.acquire()``</font>  bevor kritischer Bereich ausgeführt wird 
* 1. Thread der   <font color=#0000e6> ``lck.acquire()``</font>  ruft läuft weiter 
* Nächster Thread, der   <font color=#0000e6> ``lck.acquire()``</font>  muss warten  
* bis 1. Thread   <font color=#0000e6> ``lck.release()``</font>  ruft 
* usw.  

Damit wird sichergestellt, dass die Methode <font color=#008000> *inc()*</font>  nur von   <font color=#ff0000> **einem**</font>  Thread gleichzeitig durchlaufen wird, alle anderen sind solange blockiert.


In [ ]:
import threading 
  
class Counter(object):
    def __init__(self):
        self.num = 0
    def inc(self):
        self.num += 1

def thread_task(counter, lock):
    for _ in range(100000):
        lock.acquire()
        counter.inc()
        lock.release()
  
# global counter
counter = Counter()

# lock
lock    = threading.Lock() 

# create two threads 
t1 = threading.Thread(target = thread_task, args = (counter,lock))
t2 = threading.Thread(target = thread_task, args = (counter,lock))
  
# start threads 
t1.start() 
t2.start() 
  
# wait until threads finish their job 
t1.join() 
t2.join() 
  
# print result
print(counter.num) 


Jetzt ist das Ergebnis richtig, aber das `locking` führt zu Geschwindigkeitseinbußen.

**Aufgabe**: Überlegen sie, wie sie die relative Ausführungsgeschwindigkeit der falschen und richtigen Variante einschätzen, aber auch im Vergleich zu einer Variante mit nur einem Thread. Messen sie die Geschwindigkeit der verschiedenen Optionen (Hinweis: `%%time`) und vergleichen sie. Erklären Sie das Ergebnis.

In diesem Fall werden die beiden Threads zusammen sogar deutlich langsamer sein als ein einzelner. Programmierung mit parallelen Threads braucht also offensichtlich große Sorgfalt und Planung, um wirklich einen Geschwindigkeitsgewinn mit sich zu bringen.